In [122]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string
import os
import re
import matplotlib.pyplot as plt

In [123]:
df = pd.read_csv('Youtube01-Psy.csv') #read the CSV file

In [124]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [125]:
column = ['COMMENT_ID','AUTHOR','DATE']
df.drop(columns = column,inplace=True)
df.head()

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [126]:
df.CLASS.value_counts()

1    175
0    175
Name: CLASS, dtype: int64

In [127]:
df.shape

(350, 2)

In [128]:
df.drop_duplicates(inplace=True)
df.shape

(349, 2)

In [129]:
df.isnull().sum()

CONTENT    0
CLASS      0
dtype: int64

In [130]:
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [131]:
sent1 = "Subject: why is this"
sent2 = re.sub("Subject:","",sent1)
tokens = word_tokenize(sent1.lower())


In [132]:
def clean_txt(sent):    
    #Stripping white spaces before and after the text
    sent = sent.strip()
    #Replacing multiple spaces with a single space
    result = re.sub("\s+", " ", sent)
    result = re.sub("Subject:","",result)
    #Replacing Non-Alpha-numeric and non space charecters with nothing
    result = re.sub("[^\w\s]","",result)
    
    tokens = word_tokenize(result.lower())
    stop_updated = stopwords.words("english")  +  ["would", "could","told"]
    text = [term for term in tokens if term not in stop_updated and len(term) > 2] 
    res = " ".join(text)
    return res

In [133]:
df['ctext'] = df.CONTENT.apply(clean_txt)

In [134]:
df.head()

,CONTENT,CLASS,ctext
0,"Huh, anyway check out this you[tube] channel: ...",1,huh anyway check youtube channel kobyoshi02
1,Hey guys check out my new channel and our firs...,1,hey guys check new channel first vid monkeys m...
2,just for test I have to say murdev.com,1,test say murdevcom
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,shaking sexy ass channel enjoy
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1,watchvvtarggvgtwq check


In [135]:
X_text = df.ctext.values
y = df.CLASS.values

In [136]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size = 0.20, random_state=311)

In [137]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=941)
X_trainmat = tfidf_vectorizer.fit_transform(X_train)

In [138]:
y.sum()

175

In [139]:
y_train.sum()

140

In [140]:
X_trainmat.toarray()

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0.5, 0.5, 0.5],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

USING SVM Linear kernel

In [141]:
from sklearn.svm import SVC
classify = SVC(kernel= 'linear')
classify.fit(X_trainmat.toarray(), y_train)

SVC(kernel='linear')

In [142]:
X_testmat = tfidf_vectorizer.transform(X_test)

In [143]:
y_pred = classify.predict(X_testmat.toarray())

In [144]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [145]:
accuracy_score(y_test,y_pred)

0.8714285714285714

In [146]:
y_predtrain = classify.predict(X_trainmat.toarray())

In [147]:
accuracy_score(y_train,y_predtrain)

1.0

In [148]:
confusion_matrix(y_train,y_predtrain)

array([[139,   0],
       [  0, 140]], dtype=int64)

In [149]:
confusion_matrix(y_test,y_pred)

array([[28,  7],
       [ 2, 33]], dtype=int64)

Using Logistic Regression

In [150]:
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size = 0.20, random_state=2505)
classifier = LogisticRegression()

In [151]:
classifier.fit(X_trainmat.toarray(), y_train)

LogisticRegression()

In [152]:
y_pred = classifier.predict(X_testmat)

In [153]:
accuracy_score(y_test,y_pred)

0.5

In [154]:
confusion_matrix(y_test,y_pred)

array([[15, 16],
       [19, 20]], dtype=int64)

In [155]:
y_predtrain = classifier.predict(X_trainmat)

In [156]:
accuracy_score(y_train,y_predtrain)

0.9390681003584229

Actual project

In [157]:
df2 = pd.read_csv('comments_dataset.csv')
df2.head()

,videoId,textDisplay,textOriginal,parentId,authorDisplayName,authorProfileImageUrl,authorChannelUrl,authorChannelId,canRate,viewerRating,likeCount,publishedAt,updatedAt,commentId
0,kX3nB4PpJko,Hi,Hi,UgycNFzwlYlQB2bKNOV4AaABAg,Lebron,https://yt3.ggpht.com/VwaM-feRzjHukiZs7q845WyJ...,http://www.youtube.com/channel/UCimAkW0digxXwB...,{'value': 'UCimAkW0digxXwBLVt7p0xYg'},True,none,0,2022-11-22T16:59:08Z,2022-11-22T16:59:08Z,UgycNFzwlYlQB2bKNOV4AaABAg.9iLZx5BVUrq9ijq1yGuiBA
1,kX3nB4PpJko,WHERE CAN YOU FIND BEAST BURGER IMEED TO KMOW!...,WHERE CAN YOU FIND BEAST BURGER IMEED TO KMOW!...,UgycNFzwlYlQB2bKNOV4AaABAg,Norabaka,https://yt3.ggpht.com/ACIoNdUG5FIrQY1A64Lm2Klj...,http://www.youtube.com/channel/UC5tWZwAhxPB5So...,{'value': 'UC5tWZwAhxPB5SoatAVDvvZQ'},True,none,0,2022-11-16T21:22:28Z,2022-11-16T21:22:28Z,UgycNFzwlYlQB2bKNOV4AaABAg.9iLZx5BVUrq9iVrPNTgmYp
2,kX3nB4PpJko,❤️❤️❤️,❤️❤️❤️,UgycNFzwlYlQB2bKNOV4AaABAg,Faisal Ahmed,https://yt3.ggpht.com/ytc/AMLnZu8UcnceoPC1dZAX...,http://www.youtube.com/channel/UC7QNPKPXzNI88a...,{'value': 'UC7QNPKPXzNI88amrVKhB01Q'},True,none,1,2022-11-13T08:04:56Z,2022-11-13T08:04:56Z,UgycNFzwlYlQB2bKNOV4AaABAg.9iLZx5BVUrq9iMhkP68H6V
3,kX3nB4PpJko,IGHT,IGHT,UgycNFzwlYlQB2bKNOV4AaABAg,Minecraft Missionary,https://yt3.ggpht.com/ytc/AMLnZu8oVIzgCvBMJ2fn...,http://www.youtube.com/channel/UCwcx5cjUNflz5Q...,{'value': 'UCwcx5cjUNflz5Q7NrW4psjw'},True,none,0,2022-11-12T22:56:28Z,2022-11-12T22:56:28Z,UgycNFzwlYlQB2bKNOV4AaABAg.9iLZx5BVUrq9iLizMfi886
4,kX3nB4PpJko,Hola,Hola,UgycNFzwlYlQB2bKNOV4AaABAg,FREN,https://yt3.ggpht.com/MaUAo6TC2Rh2RaMQh3eiSUqs...,http://www.youtube.com/channel/UCK9_bFIW8l3AYN...,{'value': 'UCK9_bFIW8l3AYNyy4ErHiXg'},True,none,0,2022-11-12T22:56:27Z,2022-11-12T22:56:27Z,UgycNFzwlYlQB2bKNOV4AaABAg.9iLZx5BVUrq9iLizFW4sRH


In [158]:
del_column = ['videoId' , 'textOriginal', 'parentId', 'authorProfileImageUrl', 'authorChannelId', 'canRate', 'viewerRating', 'publishedAt', 'updatedAt', 'commentId']
df2.drop(columns = del_column,inplace = True)
df2.head()

,textDisplay,authorDisplayName,authorChannelUrl,likeCount
0,Hi,Lebron,http://www.youtube.com/channel/UCimAkW0digxXwB...,0
1,WHERE CAN YOU FIND BEAST BURGER IMEED TO KMOW!...,Norabaka,http://www.youtube.com/channel/UC5tWZwAhxPB5So...,0
2,❤️❤️❤️,Faisal Ahmed,http://www.youtube.com/channel/UC7QNPKPXzNI88a...,1
3,IGHT,Minecraft Missionary,http://www.youtube.com/channel/UCwcx5cjUNflz5Q...,0
4,Hola,FREN,http://www.youtube.com/channel/UCK9_bFIW8l3AYN...,0


In [159]:
x = df2['textDisplay']
x

0                                                       Hi
1        WHERE CAN YOU FIND BEAST BURGER IMEED TO KMOW!...
2                                                   ❤️❤️❤️
3                                                     IGHT
4                                                     Hola
                               ...                        
18357                                             I subbed
18358                                           noooo sped
18359    I won&#39;t do that challenges and I know cuz ...
18360       A twist: Try to have them blind fold as well 😂
18361                                                   Hi
Name: textDisplay, Length: 18362, dtype: object

In [160]:
x_trainmat = tfidf_vectorizer.fit_transform(x)


In [161]:
y2 = classify.predict(x_trainmat.toarray())

In [164]:
y2[150:160]

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1], dtype=int64)

In [166]:
len(y2)

18362

In [172]:
df2['spam']=y2
df2.head()

,textDisplay,authorDisplayName,authorChannelUrl,likeCount,spam
0,Hi,Lebron,http://www.youtube.com/channel/UCimAkW0digxXwB...,0,1
1,WHERE CAN YOU FIND BEAST BURGER IMEED TO KMOW!...,Norabaka,http://www.youtube.com/channel/UC5tWZwAhxPB5So...,0,1
2,❤️❤️❤️,Faisal Ahmed,http://www.youtube.com/channel/UC7QNPKPXzNI88a...,1,1
3,IGHT,Minecraft Missionary,http://www.youtube.com/channel/UCwcx5cjUNflz5Q...,0,1
4,Hola,FREN,http://www.youtube.com/channel/UCK9_bFIW8l3AYN...,0,1


In [174]:
df2.to_csv('comments_datasetwithspam.csv')

In [178]:
df2=df2.loc[df2['spam']==0]
df2.head()

,textDisplay,authorDisplayName,authorChannelUrl,likeCount,spam
6,Yes i love this guy kets go harry❤❤,CRYPTO,http://www.youtube.com/channel/UCeYsd-jG5fwHao...,0,0
24,C&#39;est un attrape nigaud...personne ne peut...,Papillon Fr,http://www.youtube.com/channel/UCkUfc6uvlt8FMy...,0,0
31,Y este doblaje😓,franco mendoza,http://www.youtube.com/channel/UCWrHfn9O0tmVDe...,0,0
33,Your in fall guys,Ava games and more,http://www.youtube.com/channel/UCToR9_mzJpSBy5...,0,0
34,you beat pewdiepie by 2 million,Генерал Гавс,http://www.youtube.com/channel/UCfDlqbVzQAtcpB...,0,0


In [181]:
df1 = df2['textDisplay']
df1.head()
df1.to_csv('commentsonly.csv')